# Evaluating the LLM on the dataset 

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/final_fewshot.csv')

# Ensure data types are correct; convert if necessary
salary_columns = ['min_salary', 'max_salary', 'block1_min_salary_fewshot', 'block1_max_salary_fewshot', 'block1_2_min_salary_fewshot', 'block1_2_max_salary_fewshot', 'block1_2_3_min_salary_fewshot', 'block1_2_3_max_salary_fewshot']
for col in salary_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [4]:
df.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_min_salary_fewshot,block1_max_salary_fewshot,...,block1_2_answer_fewshot,block1_2_3_min_salary_fewshot,block1_2_3_max_salary_fewshot,block1_2_3_answer_fewshot,block1_2_3_4_min_salary_fewshot,block1_2_3_4_max_salary_fewshot,block1_2_3_4_answer_fewshot,block1_2_3_4_5_min_salary_fewshot,block1_2_3_4_5_max_salary_fewshot,block1_2_3_4_5_answer_fewshot
0,Software Engineer .Net,ELCA Informatik AG,Festanstellung,"Über ELCA Wir sind ELCA, einer der grössten Sc...",80,100,100000,130000,90000.0,110000.0,...,(90'000) und (110'000) CHF,90000.0,110000.0,(90'000) und (110'000) CHF,90000.0,110000.0,Das Gehalt für die Position Software Engineer ...,90000.0,110000.0,Das Gehalt für die Position Software Engineer ...
1,Digital Analyst / Web Analyst,Unic AG,Festanstellung,Deine Aufgaben als Digital Analyst bei Unic ...,80,100,90000,113000,80000.0,100000.0,...,80'000 und 100'000 CHF,70000.0,90000.0,(70000) und (90000) CHF,80000.0,100000.0,80'000 - 100'000 CHF,92000.0,113000.0,92'000 - 113'000 CHF
2,ICT Supporter 100% (a),Spitex Zürich,Festanstellung,Möchtest du auch etwas bewegen und deine Arbei...,100,100,78000,96000,70000.0,90000.0,...,(60'000) und (80'000) CHF,70000.0,90000.0,(70000) und (90000) CHF,80000.0,100000.0,80'000 - 100'000 CHF,80000.0,100000.0,Das Gehalt beträgt zwischen 80'000 und 100'000...
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300,60000.0,80000.0,...,(60'000 - 80'000) CHF,45000.0,65000.0,45000 und 65000 CHF,NaN,NaN,"Ich kenne das genaue Gehalt nicht, daher schät...",NaN,NaN,2177 CHF
4,System Engineer für Microsoft-Technologien und...,konekkt GmbH,Festanstellung,Wir präsentieren eine faszinierende Karrieremö...,80,100,95000,125000,90000.0,110000.0,...,(90'000) und (110'000) CHF,90000.0,110000.0,(90'000) und (110'000) CHF,90000.0,110000.0,90'000 - 110'000 CHF,90000.0,110000.0,(90'000 - 110'000) CHF


In [5]:
def calculate_rmse(row, min_col, max_col, pred_min_col, pred_max_col):
    actual_avg = (row[min_col] + row[max_col]) / 2
    predicted_avg = (row[pred_min_col] + row[pred_max_col]) / 2
    return np.sqrt((actual_avg - predicted_avg) ** 2)

def calculate_overlap(row, min_col, max_col, pred_min_col, pred_max_col):
    actual_min, actual_max = row[min_col], row[max_col]
    predicted_min, predicted_max = row[pred_min_col], row[pred_max_col]
    intersection_min = max(actual_min, predicted_min)
    intersection_max = min(actual_max, predicted_max)
    if intersection_max > intersection_min:
        intersection = intersection_max - intersection_min
        smaller_range = min(actual_max - actual_min, predicted_max - predicted_min)
        return intersection / smaller_range
    return 0

In [6]:
blocks = [
    ('block1_min_salary_fewshot', 'block1_max_salary_fewshot'),
    ('block1_2_min_salary_fewshot', 'block1_2_max_salary_fewshot'),
    ('block1_2_3_min_salary_fewshot', 'block1_2_3_max_salary_fewshot')
]

# Iterate over each block and calculate RMSE and overlap
for min_col, max_col in blocks:
    df[min_col + '_rmse'] = df.apply(calculate_rmse, axis=1, min_col='min_salary', max_col='max_salary', pred_min_col=min_col, pred_max_col=max_col)
    df[min_col + '_overlap'] = df.apply(calculate_overlap, axis=1, min_col='min_salary', max_col='max_salary', pred_min_col=min_col, pred_max_col=max_col)

# Calculate average RMSE and overlap for each block
for min_col, _ in blocks:
    average_rmse = df[min_col + '_rmse'].mean()
    average_overlap = df[min_col + '_overlap'].mean()
    print(f"Average RMSE for {min_col}: {average_rmse}")
    print(f"Average Overlap Coefficient for {min_col}: {average_overlap}")

Average RMSE for block1_min_salary_fewshot: 19828.072727272727
Average Overlap Coefficient for block1_min_salary_fewshot: 0.43021212121212127
Average RMSE for block1_2_min_salary_fewshot: 20891.70909090909
Average Overlap Coefficient for block1_2_min_salary_fewshot: 0.396010101010101
Average RMSE for block1_2_3_min_salary_fewshot: 21209.890909090907
Average Overlap Coefficient for block1_2_3_min_salary_fewshot: 0.3737878787878788
